# ML & AL 2023 Spring Course Project
### Jiang Yuchen 12232418

In this project, we are asked to finish several tasks which aim to find the shortest path in a weighted directed graph, including dynamic programming, reinforcement learning(Q-learning) and value function approximation. We also try to train a model based on value function approximation such that the obtained solution can work even if you offer a different graph. All codes are included below. 

In [1]:
## necessary python packages
import collections
import random
import numpy as np
import time

## Task 1  Weighted directed graph
Write a python code to create a weighted directed graph with arbitrary number of
nodes and edges, for which the edges number can be modified by some parameters

## Task 2  Dynamic programming
By using dynamic programming, solve the problem of finding a shortest path fromanygiven starting node to any given target node in the weighted directed graph. In other words, you need to find out the optimal policy and optimal trajectory. The solution should still work without re-do dynamic programming when changing the starting node and target node.

We apply dynamic programming in WDGraph class, which is shortest_path_solution() method. The implementation uses the floyd_warshall algorithm to compute the shortest distances and keeps track of the previous nodes to reconstruct the path, in order to avoid re-doing dynamic programming when changing start and end node pairs. Here we construct the example to test the dynamic programming.


In [2]:
class WDGraph():
    """
    We define WDGraph as a class for weighted directed graph, which is used through the subsequent tasks.
    The class can init an instance of graph, add and delete edges.
    Also, the class can find the shortest path for any pair of start and end nodes as it is initialized. 
    """
    def __init__(self, nodes, edges):
        """
        self.graph is a dictionary to collect nodes with their connected nodes and weights
        e.g. {node:[(cost,neighbour), ...]}
        self.nodes and self.edges are lists to record all nodes and edges information.
        e.g. [node_1, node_2,...], [(from_node, to_node, weight), ...]
        self.init_graph() is called to construct the graph based on received nodes and edges
        add_edge() and delete_edge() is designed to add/delete edge in the graph
        shortest_path_fw() is designed to record global information of shortest path
        """
        self.graph = dict()
        self.nodes = nodes
        self.edges = edges
        self.node2num = {}    # easily record
        self.init_graph(nodes, edges)
        self.global_distance, self.global_path = self.shortest_path_fw()
    
    def init_graph(self, nodes, edges):
        """
        constract the graph according to nodes and edges
        """
        for i in range(len(nodes)):
            self.graph[nodes[i]] = []
            self.node2num[nodes[i]] = i
        for edge in edges:
            (from_node, to_node, weight) = edge
            self.graph[from_node].append((weight, to_node))
        
    
    def add_edge(self, from_node, to_node, weight):
        """
        add an edge to the graph, input form is "from_node, to_node and weight"
        if from_node or to_node not in node list, raise exception
        """
        if from_node in self.nodes and to_node in self.nodes:
            self.graph[from_node].append((weight, to_node))
        else:
            raise Exception("Added edge contains invalid node!")

    def delete_edge(self, from_node, to_node):
        """
        delete an edge to the graph, input form is "from_node, to_node"
        weight is not needed
        if edge is invalid, raise exception
        """
        if from_node not in self.nodes or to_node not in self.nodes:
            raise Exception("Target edge contains invalid node!")
        for i in range(len(self.graph[from_node])):
            if self.graph[from_node][i][1] == to_node:
                self.graph[from_node].pop(i)
                break
                
    def shortest_path_fw(self):
        """
        find shortest path by floyd_warshall algorithm
        You can run it by hand as edges are added or deleted to update global distance and path matrix
        """
        # cannot search and give error if the graph is not constructed
        if not self.graph:
            raise Exception("Graph is not constructed!")
            
        # Number of vertices in the graph
        n = len(self.nodes)
        
        # Initialize the distance matrix with the graph's weights
        # -1 for unreachble pair
        dist = []
        # Initialize path matrix
        path = [[None] * n for _ in range(n)]
        for i in range(n):
            dist.append([])
            for j in range(n):
                if j == i:
                    dist[-1].append(0)
                else:
                    dist[-1].append(-1)
        for node in self.nodes:
            for weight, neighbor in self.graph[node]:
                node_id, neighbor_id = self.node2num[node], self.node2num[neighbor]
                dist[node_id][neighbor_id] = weight
                path[node_id][neighbor_id] = node_id

        for i in range(len(dist)):
            for j in range(len(dist[i])):
                if dist[i][j] == -1:
                    dist[i][j] = float("inf")
        
        # Find the shortest path between all pairs of vertices
        for k in range(n):
            for i in range(n):
                for j in range(n):
                    # If vertex k is on the shortest path from i to j, update the distance
                    # record vertex k for i and j
                    if dist[i][k] + dist[k][j] < dist[i][j]:  
                        dist[i][j] = dist[i][k] + dist[k][j]
                        path[i][j] = path[k][j]

        return dist, path
    
    def get_shortest_path(self, start, end):
        """
        reconstruct the shortest path according to self.global path and start, end nodes
        """
        i, j = start, end
        if self.global_path[i][j] is None:
            return []

        path = [j]
        while i != j:
            j = self.global_path[i][j]
            if j is None:
                break
            path.append(j)

        return path[::-1]
    
    def shortest_path_solution(self, start_node, end_node):
        """
        give solution for shortest path recorded in self.dist and self.path by start and end node pair
        """
        # If receive invalid node, raise exception
        if start_node not in self.nodes or end_node not in self.nodes:
            raise Exception("Invalid node!")
            
        # turn node into id for find results
        start_index = self.node2num[start_node]
        end_index = self.node2num[end_node]
        path = self.get_shortest_path(start_index, end_index)
        
        # change index into node for review
        node_path = []
        for idx in path:
            node_path.append(self.nodes[idx])
        return self.global_distance[start_index][end_index], node_path


## Test 1
Here we design an example to test task 1 and 2. The results show that the dynamic programming is successfully done.

In [3]:
# test with given nodes and edges
nodes = ["A", "B", "C", "D", "E", "F", "G"]
edges = [
        ("A", "B", 7),
        ("A", "D", 5),
        ("B", "C", 8),
        ("B", "D", 9),
        ("B", "E", 7),
        ("C", "E", 5),
        ("D", "E", 15),
        ("D", "F", 6),
        ("E", "F", 8),
        ("E", "G", 9),
        ("F", "G", 11)
    ]
graph = WDGraph(nodes, edges)

In [4]:
graph.shortest_path_fw()

([[0, 7, 15, 5, 14, 11, 22],
  [inf, 0, 8, 9, 7, 15, 16],
  [inf, inf, 0, inf, 5, 13, 14],
  [inf, inf, inf, 0, 15, 6, 17],
  [inf, inf, inf, inf, 0, 8, 9],
  [inf, inf, inf, inf, inf, 0, 11],
  [inf, inf, inf, inf, inf, inf, 0]],
 [[None, 0, 1, 0, 1, 3, 5],
  [None, None, 1, 1, 1, 3, 4],
  [None, None, None, None, 2, 4, 4],
  [None, None, None, None, 3, 3, 5],
  [None, None, None, None, None, 4, 4],
  [None, None, None, None, None, None, 5],
  [None, None, None, None, None, None, None]])

In [5]:
# Review the constructed graph
graph.graph

{'A': [(7, 'B'), (5, 'D')],
 'B': [(8, 'C'), (9, 'D'), (7, 'E')],
 'C': [(5, 'E')],
 'D': [(15, 'E'), (6, 'F')],
 'E': [(8, 'F'), (9, 'G')],
 'F': [(11, 'G')],
 'G': []}

In [6]:
# Give start and end node, find shortest path
start_node, end_node = "C", "F"
graph.shortest_path_solution(start_node, end_node)

(13, ['C', 'E', 'F'])

In [8]:
# Can be used for multiple times
start_node, end_node = "A", "G"
graph.shortest_path_solution(start_node, end_node)

(22, ['A', 'D', 'F', 'G'])

In [9]:
# Return float("inf") when unreachble
start_node, end_node = "C", "A"
graph.shortest_path_solution(start_node, end_node)

(inf, [])

## Task 3  Reinforcement learning (Q-learning)
Solve the above problem by using one of these reinforcement learning techniques, such as MC, SARSA(0), SARSA($\lambda$), Q-learning.

## Task 4 Value function approximation
Considering value function approximation, solve the above problem.

Here we use Q-learning to solve the problem. The Q-values are iteratively updated based on the rewards obtained during training episodes. The `get_epsilon_greedy_action` function implements the epsilon-greedy policy for action selection.

Besides, the value function approximation method is included in `RLShortestPath` class. The code demonstrates the usage of Value Function Approximation (VFA) to find the shortest path from a given starting node to a target node in the weighted directed graph. It uses a linear approximation function with weights to estimate the value of a state. The weights are updated through gradient descent to minimize the prediction error.

In [19]:
class RLShortestPath:
    def __init__(self, graph):
        self.G = graph
        self.R = {}    # record weight information for reward setting
        self.q_values = self.init_q_values()    # apply q learning
        self.weights = self.init_weights()     # apply value function approximation
        self.node2num = {}    # easily record, used in VFA
        for i in range(len(graph.nodes)):
            self.node2num[graph.nodes[i]] = i
        self.q_ans = self.q_learning()    # record global shortest path answer by q-learning
        self.vfa_ans = self.VFA()    # record global shortest path answer by value function approximation
        self.q_time_cost, self.vfa_time_cost = 0, 0    # record time cost, since main process happen in __init__
    
    def path2cost(self, path):
        """
        count cost on given path generated from Q-learning and VFA
        """
        cost = 0
        curr_node = path[0]
        for i in range(1, len(path)):
            to_node = path[i]
            for weight, neighbor in self.G.graph[curr_node]:
                if neighbor == to_node:
                    cost += weight
                    break
            curr_node = path[i]
        return cost

    """
    Q-learning Part:
    1. init q values for each node
    2. train q learning for start node and end node
    3. update q value during training according to epsilon greedy action
    4. find shortest path according to updated q values
    """
    def init_q_values(self):
        """
        all q values are set to 0 for reachable pair
        record weight information by the way
        """
        q_values = {}
        for node in self.G.nodes:
            q_values[node] = {}
            for weight, neighbor in self.G.graph[node]:
                q_values[node][neighbor] = 0
                pair = node+neighbor
                self.R[pair] = weight
        return q_values

    def train_q_learning(self, start_node, target_node, num_episodes=1500, learning_rate=0.2, discount_factor=0.9):
        """
        main part for training Q-learning
        set num_episodes, learning_rate and discount_factor for training
        """
        # in order to avoid possible cycle in graph which leads to infinite loop, count duplicate visited node
        # as duplicated visit time is larger than threshold, break the loop
        cycle_threshold = 5
        for episode in range(num_episodes):
            current_node = start_node
            cycle_detect = []
            detect_times = 0
            #  The Q-values are iteratively updated based on the rewards obtained during training episodes. 
            #  The get_epsilon_greedy_action function implements the epsilon-greedy policy for action selection.
            while current_node != target_node:
                if not self.q_values[current_node]:
                    break
                cycle_detect.append(current_node)
                action = self.get_epsilon_greedy_action(current_node)
                next_node = action
                # set reward as negative of edge weight and 1-weight if next node is end node, which aims for shortest path
                reward = 1-self.R[current_node+next_node] if next_node == target_node else -self.R[current_node+next_node]
                self.update_q_value(current_node, action, next_node, reward, learning_rate, discount_factor)
                current_node = next_node
                # cycle detection
                if current_node in cycle_detect:
                    detect_times += 1
                if detect_times > cycle_threshold:
                    break

    def get_epsilon_greedy_action(self, node, epsilon=0.2):
        """
        Implements the epsilon-greedy policy for action selection.
        """
        if random.random() < epsilon:
            (weight, action) = random.choice(list(self.G.graph[node]))
            return action
        else:
            return max(self.q_values[node], key=self.q_values[node].get)

    def update_q_value(self, current_node, action, next_node, reward, learning_rate, discount_factor):
        """
        update q values based on equation
        """
        if not self.q_values[next_node]:
            max_q_value = 0
        else:
            max_q_value = max(self.q_values[next_node].values())
        self.q_values[current_node][action] += learning_rate * (reward + 
                    discount_factor * max_q_value - self.q_values[current_node][action])

    def shortest_path_based_on_q_value(self, start_node, target_node):
        """
        find shortest path based on updated q values
        from start node, we choose each action with maximum q value
        return infinite if unreachble
        """
        cycle_threshold = 5
        detect_times = 0
        cycle_detect = []
        current_node = start_node
        path = [current_node]
        while current_node != target_node:
            if not self.q_values[current_node]:
                return float("inf")
            cycle_detect.append(current_node)
            action = max(self.q_values[current_node], key=self.q_values[current_node].get)
            next_node = action
            path.append(next_node)
            current_node = next_node
            if current_node in cycle_detect:
                detect_times += 1
            if detect_times > cycle_threshold:
                return float("inf")
        return path
    
    def q_learning(self):
        """
        The main part of Q-learning
        The single pair of start and end node in Q-learning is described above
        In order to maintain whole graph, run init_q_values and train_q_learning for all possible pair
        """
        q_start_time = time.time()
        ans = {}
        for start in self.G.nodes:
            for end in self.G.nodes:
                self.q_values = self.init_q_values()
                self.train_q_learning(start, end)
                path = self.shortest_path_based_on_q_value(start, end)
                pair = start + end
                # if reachble, get the cost of the path
                if path != float("inf"):
                    ans[pair] = (path, self.path2cost(path))
                else:
                    ans[pair] = ([], float("inf"))
        self.q_time_cost = time.time() - q_start_time
        print("Time cost for Q-Learning:", self.q_time_cost, "s")    # output q-learning time cost
        return ans
    
    def q_learning_solution(self, start, end):
        """
        Get solution from q-learning's answer
        """
        return self.q_ans[start+end]
        
    """
    Value Function Approximation Part:
    1. design features and init weights for value function
    2. train value function by episodes
    3. update weights during training
    4. find shortest path by VFA
    """
    def init_weights(self):
        num_features = len(self.G.graph) * 2  # Features: node visited (0/1) for each node
        weights = np.zeros(num_features)
        return weights
    
    def get_features(self, node, visited_nodes):
        num_nodes = len(self.G.graph)
        features = np.zeros(num_nodes * 2)
        node_id = self.node2num[node]
        features[node_id] = 1
        features[num_nodes + node_id] = visited_nodes.count(node)
        return features
    
    def value_function(self, state):
        return np.dot(self.weights, state)
    
    def update_weights(self, state, target, learning_rate):
        prediction = self.value_function(state)
        error = target - prediction
        self.weights += learning_rate * error * state
    
    def train_value_function(self, start_node, target_node, num_episodes=1500, learning_rate=0.1, discount_factor=0.9):
        # also detect possible cycle
        cycle_threshold = 5
        for episode in range(num_episodes):
            current_node = start_node
            visited_nodes = []
            detect_times = 0
            cycle_detect = []
            while current_node != target_node:
                visited_nodes.append(current_node)
                state = self.get_features(current_node, visited_nodes)
                if not self.G.graph[current_node]:
                    break
                cycle_detect.append(current_node)
                next_node = random.choice([neighbor for _, neighbor in self.G.graph[current_node]])
                reward = 1 if next_node == target_node else -1
                next_state = self.get_features(next_node, visited_nodes)
                target = reward + discount_factor * self.value_function(next_state)
                self.update_weights(state, target, learning_rate)
                current_node = next_node
                # cycle detection
                if current_node in cycle_detect:
                    detect_times += 1
                if detect_times > cycle_threshold:
                    break
    
    def shortest_path_based_on_VFA(self, start_node, target_node):
        cycle_threshold = 5
        detect_times = 0
        cycle_detect = []
        current_node = start_node
        path = [current_node]
        while current_node != target_node:
            visited_nodes = path
            state = self.get_features(current_node, visited_nodes)
            if not self.G.graph[current_node]:
                return float("inf")
            cycle_detect.append(current_node)
            next_node = random.choice([neighbor for _, neighbor in self.G.graph[current_node]])
            path.append(next_node)
            current_node = next_node
            if current_node in cycle_detect:
                detect_times += 1
            if detect_times > cycle_threshold:
                return float("inf")
        return path
    
    def VFA(self):
        vfa_start_time = time.time()
        ans = {}
        for start in self.G.nodes:
            for end in self.G.nodes:
                self.weights = self.init_weights()
                self.train_value_function(start, end)
                path = self.shortest_path_based_on_VFA(start, end)
                pair = start + end
                if path != float("inf"):
                    ans[pair] = (path, self.path2cost(path))
                else:
                    ans[pair] = ([], float("inf"))
        self.vfa_time_cost = time.time() - vfa_start_time
        print("Time cost for Value Function Approximation:", self.vfa_time_cost, "s")
        return ans
    
    def VFA_solution(self, start, end):
        return self.vfa_ans[start+end]

## Task 2
Also, we design the given exmaple and an random example to compare dynamic programming, Q-learning and Value Function Approximation methods. Detail analysis is described in report.

### test with given nodes and edges

In [63]:
nodes = ["A", "B", "C", "D", "E", "F", "G"]
edges = [
        ("A", "B", 7),
        ("A", "D", 5),
        ("B", "C", 8),
        ("B", "D", 9),
        ("B", "E", 7),
        ("C", "E", 5),
        ("D", "E", 15),
        ("D", "F", 6),
        ("E", "F", 8),
        ("E", "G", 9),
        ("F", "G", 11)
    ]

In [64]:
# Dynamic Programming
start_time = time.time()
graph = WDGraph(nodes, edges)
time_cost = time.time() - start_time
print("Time cost:", time_cost, "s")

Time cost: 0.0004220008850097656 s


In [65]:
# Q-learning and VFA
start_time = time.time()
rl = RLShortestPath(graph)
time_cost = time.time() - start_time
print("Time cost:", time_cost, "s")

Time cost for Q-Learning: 0.19626808166503906 s
Time cost for Value Function Approximation: 1.2574656009674072 s
Time cost: 1.4540777206420898 s


In [66]:
graph.graph

{'A': [(7, 'B'), (5, 'D')],
 'B': [(8, 'C'), (9, 'D'), (7, 'E')],
 'C': [(5, 'E')],
 'D': [(15, 'E'), (6, 'F')],
 'E': [(8, 'F'), (9, 'G')],
 'F': [(11, 'G')],
 'G': []}

In [69]:
s, e = "A", "F"
print(graph.shortest_path_solution(s, e))
print(rl.q_learning_solution(s, e))
print(rl.VFA_solution(s, e))

(11, ['A', 'D', 'F'])
(['A', 'D', 'F'], 11)
(['A', 'D', 'F'], 11)


In [73]:
s, e = "G", "A"
print(graph.shortest_path_solution(s, e))
print(rl.q_learning_solution(s, e))
print(rl.VFA_solution(s, e))

(inf, [])
([], inf)
([], inf)


In [44]:
# randomly generate graph
def generate_graph(num_nodes=20):
    nodes = random.sample(range(1, num_nodes+1), num_nodes)

    edges_num = random.randint(num_nodes*2, num_nodes*5)    # set number of edges
    edges_nodes = []
    edges_weights = []
    for _ in range(edges_num):
        # randomly add edge which is not duplicate
        from_node, to_node = random.sample(nodes, 2)
        while (from_node, to_node) in edges_nodes:
            from_node, to_node = random.sample(nodes, 2)
        edges_nodes.append((from_node, to_node))
        edges_weights.append(random.randint(1, 30))
    edges = []
    for i in range(edges_num):
        edges.append((str(edges_nodes[i][0]), str(edges_nodes[i][1]), int(edges_weights[i])))
    for i in range(len(nodes)):
        nodes[i] = str(nodes[i])
    return nodes, edges

Test with 10 nodes

In [45]:
nodes, edges = generate_graph(num_nodes=10)

In [46]:
# Dynamic Programming
start_time = time.time()
graph = WDGraph(nodes, edges)
time_cost = time.time() - start_time
print("Time cost:", time_cost, "s")

Time cost: 0.0009260177612304688 s


In [47]:
# Q-learning and VFA
start_time = time.time()
rl = RLShortestPath(graph)
time_cost = time.time() - start_time
print("Time cost:", time_cost, "s")

Time cost for Q-Learning: 0.9140703678131104 s
Time cost for Value Function Approximation: 10.919919729232788 s
Time cost: 11.835331916809082 s


In [48]:
graph.graph

{'5': [(23, '2'), (17, '1'), (6, '7'), (17, '9'), (27, '10')],
 '7': [(28, '2'), (5, '9'), (21, '1')],
 '1': [(8, '6'), (7, '4'), (22, '8')],
 '10': [(6, '7'), (3, '8'), (26, '4')],
 '9': [(8, '4'), (28, '7'), (8, '1'), (21, '6')],
 '8': [(8, '9')],
 '3': [(26, '6'), (9, '4'), (1, '5')],
 '6': [(15, '10'), (19, '2')],
 '2': [(24, '4'), (26, '9')],
 '4': [(17, '1'), (16, '10'), (13, '3'), (30, '8'), (4, '7'), (15, '5')]}

In [49]:
s, e = "1", "7"
print(graph.shortest_path_solution(s, e))
print(rl.q_learning_solution(s, e))
print(rl.VFA_solution(s, e))

(11, ['1', '4', '7'])
(['1', '4', '7'], 11)
(['1', '4', '3', '4', '8', '9', '7'], 95)


In [26]:
s, e = "1", "8"
print(graph.shortest_path_solution(s, e))
print(rl.q_learning_solution(s, e))
print(rl.VFA_solution(s, e))

(inf, [])
([], inf)
([], inf)


Test with 40 nodes

In [74]:
nodes, edges = generate_graph(num_nodes=40)

# Dynamic Programming
start_time = time.time()
graph = WDGraph(nodes, edges)
time_cost = time.time() - start_time
print("Time cost:", time_cost, "s")

# Q-learning and VFA
start_time = time.time()
rl = RLShortestPath(graph)
time_cost = time.time() - start_time
print("Time cost:", time_cost, "s")

Time cost: 0.017249345779418945 s
Time cost for Q-Learning: 33.68354105949402 s
Time cost for Value Function Approximation: 465.76904702186584 s
Time cost: 499.4538698196411 s


In [76]:
s, e = "2", "7"
print(graph.shortest_path_solution(s, e))
print(rl.q_learning_solution(s, e))
print(rl.VFA_solution(s, e))

(39, ['2', '8', '34', '30', '32', '17', '7'])
(['2', '8', '34', '30', '32', '17', '7'], 39)
([], inf)


# Task 5 
Train a model based on value function approximation such that the obtained solutioncan work even if you offer a different graph. In other words, the obtained solution canbeapplied to different weighted directed graphs without re-training.

Here I just provide a possbile model but have no time for furthre training.

The `ValueFunctionApproximator` class provide the VFA model for different weighted directed graphs without re-training. We simply design full-connected layers and use `ReLU` function as activation function. We need a dataset which contains enough weighted directed graphs for training.

In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class ValueFunctionApproximator(nn.Module):
    def __init__(self, num_features):
        super(ValueFunctionApproximator, self).__init__()
        self.fc1 = nn.Linear(num_features, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class ShortestPathVFA:
    def __init__(self, graph):
        self.graph = graph
        self.model = None

    def initialize_model(self, num_features):
        self.model = ValueFunctionApproximator(num_features)

    def get_features(self, node, visited_nodes):
        num_nodes = len(self.graph)
        features = np.zeros(num_nodes * 2)
        features[node] = 1
        features[num_nodes + node] = visited_nodes.count(node)
        return features

    def value_function(self, state):
        state_tensor = torch.from_numpy(state).float()
        return self.model(state_tensor).item()

    def update_model(self, states, targets, learning_rate):
        states_tensor = torch.from_numpy(states).float()
        targets_tensor = torch.from_numpy(targets).float()
        criterion = nn.MSELoss()
        optimizer = optim.SGD(self.model.parameters(), lr=learning_rate)
        optimizer.zero_grad()
        outputs = self.model(states_tensor)
        loss = criterion(outputs, targets_tensor)
        loss.backward()
        optimizer.step()

    def train_value_function(self, start_node, target_node, num_episodes=100, learning_rate=0.1, discount_factor=0.9):
        num_nodes = len(self.graph)
        num_features = num_nodes * 2
        self.initialize_model(num_features)

        for episode in range(num_episodes):
            current_node = start_node
            visited_nodes = []
            states = []
            targets = []
            while current_node != target_node:
                visited_nodes.append(current_node)
                state = self.get_features(current_node, visited_nodes)
                next_node = self.get_next_node(current_node)
                reward = self.get_reward(next_node, target_node)
                next_state = self.get_features(next_node, visited_nodes)
                target = reward + discount_factor * self.value_function(next_state)
                states.append(state)
                targets.append(target)
                current_node = next_node

            self.update_model(np.array(states), np.array(targets), learning_rate)

    def get_next_node(self, current_node):
        neighbors = [neighbor for neighbor, _ in self.graph[current_node]]
        return np.random.choice(neighbors)

    def get_reward(self, next_node, target_node):
        return 1 if next_node == target_node else -1

    def shortest_path(self, start_node, target_node):
        current_node = start_node
        path = [current_node]
        while current_node != target_node:
            visited_nodes = path
            state = self.get_features(current_node, visited_nodes)
            state_tensor = torch.from_numpy(state).float()
            action_value = self.model(state_tensor).detach().numpy()
            next_node = self.get_next_node(current_node)
            path.append(next_node)
            current_node = next_node
        return path


/home/yuchen/anaconda3/envs/tte/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
